In [1]:
#! pip install plotly-express
#! pip install streamlit
#! pip install pandas
#!pip install --upgrade numpy
#!pip install --upgrade openpyxl


In [2]:
import pandas as pd

import numpy as np

In [3]:

print("Version de pandas:", pd.__version__)

Version de pandas: 2.0.3


In [15]:
def read_wears():
    """Read wears tabs into wears_df"""
    excel_file='Copy of Silly_Fragrance_excel.xlsx'
    # Read all sheets into a dictionary
    all_sheets = pd.read_excel(excel_file, sheet_name=None)

    # Filter sheets containing 'Wears for'
    wears_sheets = {sheet_name: sheet_df for sheet_name, sheet_df in all_sheets.items() if 'Wears for' in sheet_name}
    
    #sort ascending, so that is in order by each year 2021-2022, 2023, 2024
    wears_sheets_ordered = dict(sorted(wears_sheets.items()))
    
    # Concatenate all DataFrames into one DataFrame with a new column 'sheet_name'
    wears_df = pd.concat([df.assign(sheet_name=sheet_name) for sheet_name, df in wears_sheets_ordered.items()], ignore_index=True)
    wears_df=wears_df.dropna(how='all')
    wears_df = wears_df.loc[:, ~wears_df.columns.str.contains('^Unnamed')]
    wears_df = wears_df[wears_df['Wears'] != 0]
    return wears_df


In [21]:
def sum_wears(wears_df,frag):
    """For wears_df and given frag, return sum of alltimewears for given frag, and dict of wears per year(per tab)"""
    
    wears_allsum=wears_df.groupby(['Fragrance'])['Wears'].sum()
    
    frag_wears_alltime=wears_allsum[frag]
    rank= int(wears_allsum.rank(ascending=False)[frag])
    
    total_ml_df=wears_df[wears_df["Fragrance"]==frag]["mL"] * wears_df[wears_df["Fragrance"]==frag]["Backups"]
    total_ml=total_ml_df.unique()

    # if total_ml no consensus amongst Wears tabs, take the last one -1 (most recent). 
    # if consensus, 0 to grab the only value
    # assume 12 sprays per mL. 4 sprays per wear. So 1 wear is 4/12 of a mL
    if len(total_ml) >1:
        leftover_ml=total_ml[-1]-(frag_wears_alltime*4/12)
    else:
        leftover_ml=total_ml[0]-(frag_wears_alltime*4/12)


    return frag_wears_alltime,rank, leftover_ml,wears_df[wears_df["Fragrance"]==frag]



In [22]:
wears_df=read_wears()
frag="Iris Poudre"
sum_wears(wears_df,frag)

(60,
 1,
 180.0,
       Fragrance           House   mL       Nose  \
 0   Iris Poudre  Frederic Malle  200  P.Bourdon   
 53  Iris Poudre  Frederic Malle  200  P.Bourdon   
 99  Iris Poudre  Frederic Malle  200  P.Bourdon   
 
                                      Notes  Wears  Backups  \
 0   Iris, aldehydes, yellow florals, tonka     41        1   
 53  Iris, aldehydes, yellow florals, tonka     18        1   
 99  Iris, aldehydes, yellow florals, tonka      1        1   
 
              sheet_name  
 0   Wears for 2021-2022  
 53       Wears for 2023  
 99       Wears for 2024  )

In [23]:
%run -i wears.ipynb


Version de pandas: 2.0.3
